In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle 

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from infrasim.optimise import *
from infrasim.utils import *


# save
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)


#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'


# Params
timesteps=None#7000
super_source=False
pprint=True
save_figures=True

infrasim_init_directories()

scenarios = {#'BAS' : False,
             #'BAU' : True,
             'NCO' : True,
             #'EAG' : True,
             #'COO' : True,
             #'UTO' : True,
            }

results = {}
for s in scenarios:
        
    model_run = nextra(nodes,edges,flows,
                       scenario=s,
                       energy_objective=scenarios[s],
                       timesteps=timesteps,
                       #super_source=True,
                       #super_sink=True,
                       #res_factor=99,
                       #model_name='meow',
                      )

    model_run.build()
    model_run.run(pprint=False)
    try:
        model_results = model_run.get_results()
        # add scenarios to results
        if s == 'BAU' and scenarios[s] == False:
            s = 'BAS'
        model_results.results_capacities['scenario']       = s
        model_results.results_storages['scenario']         = s
        model_results.results_edge_flows['scenario']       = s
        model_results.results_capacity_change['scenario']  = s
        model_results.results_costs['scenario']            = s
        # append results
        results[s] = model_results
        print('> Completed: ' + s)
    except:
        print('> FAILED! ' + s)


save_object(results, '../outputs/results/model_run_results.pkl')

print('> Done.')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-03



In [ ]:
model_results.plot_hourly_profile(territory='Israel',day=1,month=6)

In [ ]:
model_results.get_regional_capacity()

In [ ]:
model_results.get_regional_capacity_change()

In [ ]:
def plot_hourly_profile(self,day,month,year=2030,territory=None,**kwargs):
    '''Plot hourly profile of energy supply for a given day
    '''
    # index for day
    idx = self.results_edge_flows.loc[(self.results_edge_flows.day == day) & \
                                    (self.results_edge_flows.month == month) & \
                                    (self.results_edge_flows.year == year)].reset_index(drop=True)
    # groupby hour
    idx = idx.groupby(by=['from_id','hour']).sum().reset_index()
    # drop generation nodes
    idx = idx.loc[~idx.from_id.str.contains('generation')].reset_index(drop=True)
    # map tech and territory
    idx = map_attributes(self,idx)
    if territory is None:
        # sum across all regions
        idx = idx.groupby(by=['technology','hour']).sum().reset_index()
    else:
        # index by territory
        idx = idx.loc[idx.territory.isin([territory])].reset_index(drop=True)
    # reindex
    idx = idx[['hour','value','technology']]
    # battery charging/discharging
    idx.loc[(idx.value <= 0) & (idx.technology == 'Battery'),'technology'] = 'Battery Discharge'
    idx.loc[(idx.value > 0) & (idx.technology == 'Battery'),'technology'] = 'Battery Charge'
    # pivot table
    idx = idx.pivot_table(columns='technology',index='hour',values='value')
    # plot
    idx.plot.area(ax=kwargs.get("ax", None),
                  #cmap=kwargs.get("cmap", 'YlGnBu'),
                  color=[technology_color_dict.get(x, '#333333') for x in idx.columns],
                  linewidth=kwargs.get("linewidth", 0),
                  alpha=kwargs.get("alpha", 1),)